In [21]:
import requests
import json
from datetime import date, timedelta, datetime
from dateutil import parser
import sys

In [22]:
study_id = 'StudyHack'
ae_domain_id ='ae'
cm_domain_id = 'cm'
base_url = 'https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/'

In [23]:
ae_subject_id_url = base_url + study_id+'/'+ae_domain_id+'/subject/list'
cm_subject_id_url = base_url + study_id+'/'+cm_domain_id+'/subject/list'

In [24]:
print(ae_subject_id_url)

https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/list


In [25]:
print(cm_subject_id_url)

https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/list


In [26]:
try:
    ae_response_data1 = requests.get(ae_subject_id_url)
    ae_response_value1 = ae_response_data1.json()
    ae_subject_ids = ae_response_value1["data"]
except:
    sys.exit("Unable to get the values!")

In [27]:
try:
    cm_response_data1 = requests.get(cm_subject_id_url)
    cm_response_value1 = cm_response_data1.json()
    cm_subject_ids = cm_response_value1["data"]
except:
    sys.exit("Unable to get the values!")

In [30]:
# Patient and row for which Medication is given prior to Adverse Event
# Function will get data at subject_id level 
# Identify discrepancy and return discrepancy_list as a list 

def type1_check(ae_subject_data,cm_subject_data):
    discrepancy_list = []
    
    for cm_row in cm_subject_data:
        #cm_data = cm_row.json()
        cmaeno = cm_row["cmaeno"]
        cmstdat = cm_row["cmstdat"]
        cmendat = cm_row["cmendat"]
        cmtrt = cm_row["cmtrt"]
        
        for ae_row in ae_subject_data:
            #ae_data = ae_row.json()
            aespid = ae_row["aespid"]
            aestdat = ae_row["aestdat"]
            aeendat = ae_row["aeendat"]
            aeterm = ae_row["aeterm"]
            
            if aespid == cmaeno:
                if cmstdat < aestdat:
                    payload={}
                    payload['formname'] = ae_row["formname"]
                    payload['formid'] = ae_row["formid"]
                    payload['formidx']= ae_row["formidx"]
                    payload['type'] = 'TYPE1'
                    payload['subjectid'] = ae_row["subjid"]
                    discrepancy_list.append(payload)
            
    return discrepancy_list

In [31]:
discrepancy_master_list = []

for subject in ae_subject_ids:
    
    ae_subject_data_url = base_url + "/" + study_id+"/" + ae_domain_id + "/subject/" + str(subject)+ "/list"
    ae_response_data2 = requests.get(ae_subject_data_url)
    ae_response_value2 = ae_response_data2.json() 
    ae_subject_data = ae_response_value2["data"]
    
    cm_subject_data_url = base_url + "/" + study_id+"/" + cm_domain_id + "/subject/" + str(subject)+ "/list"
    cm_response_data2 = requests.get(cm_subject_data_url)
    cm_response_value2 = cm_response_data2.json() 
    cm_subject_data = cm_response_value2["data"]
    
    discrepancy_list = type1_check(ae_subject_data,cm_subject_data)

print(discrepancy_list)

ConnectionError: HTTPSConnectionPool(host='pyhack-dot-pharmanlp-177020.uc.r.appspot.com', port=443): Max retries exceeded with url: /api/1//StudyHack/ae/subject/166621/list (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000023F7F444E80>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))